In [7]:
import requests
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
import psycopg2
from psycopg2 import Error


url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
import json
import os

load_dotenv()
rapidapikey = os.getenv('API_KEY')
load_dotenv()
host = os.getenv('HOST1')
db = os.getenv('DATABASE2')
port = os.getenv('PORT1')
username = os.getenv('USERNAME1')
password = os.getenv('PASSWORD1')
print(db)
print(port)
print(rapidapikey)

querystring = {"league":"39","season":"2020"}

headers = {
    "X-RapidAPI-Key": rapidapikey,
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())
def test():
    if response.status_code == 200:
        data = response.json()
        first = data["response"] [0]
        print(json.dumps(first, indent=4))
        return "worked"
    else:
        print("[ERROR] Failed to retrieve data from the API...")
        return "error"
    

football_stats
5432
3d601748d3msh063ce66c68f6389p185c7djsn1b5539649fd5


In [8]:
def get_top_scorers(url, headers, params):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as http_error_message:
        print (f"❌ [HTTP ERROR]: {http_error_message}")

    except requests.exceptions.ConnectionError as connection_error_message:
        print (f"❌ [CONNECTION ERROR]: {connection_error_message}")

    except requests.exceptions.Timeout as timeout_error_message:
        print (f"❌ [TIMEOUT ERROR]: {timeout_error_message}")

    except requests.exceptions.RequestException as other_error_message:
        print (f"❌ [UNKNOWN ERROR]: {other_error_message}")

In [5]:
get_top_scorers(url,headers,querystring)

{'get': 'players/topscorers',
 'parameters': {'league': '39', 'season': '2020'},
 'errors': [],
 'results': 20,
 'paging': {'current': 1, 'total': 1},
 'response': [{'player': {'id': 184,
    'name': 'H. Kane',
    'firstname': 'Harry Edward',
    'lastname': 'Kane',
    'age': 31,
    'birth': {'date': '1993-07-28', 'place': 'London', 'country': 'England'},
    'nationality': 'England',
    'height': '188 cm',
    'weight': '86 kg',
    'injured': False,
    'photo': 'https://media.api-sports.io/football/players/184.png'},
   'statistics': [{'team': {'id': 47,
      'name': 'Tottenham',
      'logo': 'https://media.api-sports.io/football/teams/47.png'},
     'league': {'id': 39,
      'name': 'Premier League',
      'country': 'England',
      'logo': 'https://media.api-sports.io/football/leagues/39.png',
      'flag': 'https://media.api-sports.io/flags/gb.svg',
      'season': 2020},
     'games': {'appearences': 35,
      'lineups': 35,
      'minutes': 3087,
      'number': None,
 

In [7]:
a = get_top_scorers(url,headers,querystring)["response"]
print(a[0]["player"]["name"])

H. Kane


In [9]:
def process_top_scorers(data):
    """
    Parse the JSON data required for the top scorers 
    """
    top_scorers = []
    for scorer_data in data['response']:
        statistics = scorer_data['statistics'][0]

        # Set up constants for processing data 
        player = scorer_data['player']
        player_name = player['name']
        club_name = statistics['team']['name']
        total_goals = int(statistics['goals']['total'])
        penalty_goals = int(statistics['penalty']['scored'])
        assists = int(statistics['goals']['assists']) if statistics['goals']['assists'] else 0
        matches_played = int(statistics['games']['appearences'])
        minutes_played = int(statistics['games']['minutes'])
        dob = datetime.strptime(player['birth']['date'], '%Y-%m-%d')
        age = (datetime.now() - dob).days // 365

        # Append data 
        top_scorers.append({
            'player': player_name,
            'club': club_name,
            'total_goals': total_goals,
            'penalty_goals': penalty_goals,
            'assists': assists,
            'matches': matches_played,
            'mins': minutes_played,
            'age': age
        })
    return top_scorers

In [20]:
process_top_scorers(get_top_scorers(url,headers,querystring))

[{'player': 'H. Kane',
  'club': 'Tottenham',
  'total_goals': 23,
  'penalty_goals': 4,
  'assists': 14,
  'matches': 35,
  'mins': 3087,
  'age': 30},
 {'player': 'Mohamed Salah',
  'club': 'Liverpool',
  'total_goals': 22,
  'penalty_goals': 6,
  'assists': 5,
  'matches': 37,
  'mins': 3082,
  'age': 31},
 {'player': 'Bruno Fernandes',
  'club': 'Manchester United',
  'total_goals': 18,
  'penalty_goals': 9,
  'assists': 12,
  'matches': 37,
  'mins': 3109,
  'age': 29},
 {'player': 'Son Heung-Min',
  'club': 'Tottenham',
  'total_goals': 17,
  'penalty_goals': 1,
  'assists': 10,
  'matches': 37,
  'mins': 3126,
  'age': 31},
 {'player': 'P. Bamford',
  'club': 'Leeds',
  'total_goals': 17,
  'penalty_goals': 2,
  'assists': 7,
  'matches': 38,
  'mins': 3062,
  'age': 30},
 {'player': 'D. Calvert-Lewin',
  'club': 'Everton',
  'total_goals': 16,
  'penalty_goals': 0,
  'assists': 0,
  'matches': 33,
  'mins': 2876,
  'age': 27},
 {'player': 'J. Vardy',
  'club': 'Leicester',
  't

In [10]:
def create_dataframe(top_scorers):
    df = pd.DataFrame(top_scorers)
    df.sort_values(by=['total_goals', 'assists'], ascending=[False, False], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['position'] = df['total_goals'].rank(method='dense', ascending=False).astype(int)
    df = df[['position', 'player', 'club', 'total_goals', 'penalty_goals', 'assists', 'matches', 'mins', 'age']]
    return df

In [22]:
create_dataframe(process_top_scorers(get_top_scorers(url,headers,querystring)))

,position,player,club,total_goals,penalty_goals,assists,matches,mins,age
0,1,H. Kane,Tottenham,23,4,14,35,3087,30
1,2,Mohamed Salah,Liverpool,22,6,5,37,3082,31
2,3,Bruno Fernandes,Manchester United,18,9,12,37,3109,29
3,4,Son Heung-Min,Tottenham,17,1,10,37,3126,31
4,4,P. Bamford,Leeds,17,2,7,38,3062,30
5,5,D. Calvert-Lewin,Everton,16,0,0,33,2876,27
6,6,J. Vardy,Leicester,15,8,9,34,2845,37
7,7,O. Watkins,Aston Villa,14,1,5,37,3329,28
8,8,İ. Gündoğan,Manchester City,13,1,2,28,2031,33
9,8,A. Lacazette,Arsenal,13,3,2,31,1930,32


In [11]:

def create_db_connection(host_name, user_name, user_password, db_name):
    """
    Establish a connection to the PostgreSQL database
    """
    db_connection = None
    try:
        
        db_connection = psycopg2.connect(
            database="football_stats", user='postgres', password='admin', host='localhost', port= '5432'
        )
        print("PostgreSQL Database connection successful ✅")

    except Error as e:
        print(f"❌ [DATABASE CONNECTION ERROR]: '{e}'")

    return db_connection

In [54]:
create_db_connection(host,username,password,db)

PostgreSQL Database connection successful ✅


<connection object at 0x000001F596B9C040; dsn: 'user=postgres password=xxx dbname=postgres host=localhost', closed: 0>

In [12]:
def create_table(db_connection):
    CREATE_TABLE_SQL_QUERY = """
    CREATE TABLE IF NOT EXISTS top_scorers (
        position INT,
        player VARCHAR(255),
        club VARCHAR(255),
        total_goals INT,
        penalty_goals INT,
        assists INT,
        matches INT,
        mins INT,
        age INT,
        PRIMARY KEY (player, club)
    );
    """
    try:
        conn = db_connection
        cursor = conn.cursor()
        cursor.execute(CREATE_TABLE_SQL_QUERY)
        conn.commit()

    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [13]:
def insert_into_table(db_connection, df):
    """
    Insert or update the top scorers data in the database from the DataFrame
    """
    cursor = db_connection.cursor()

    INSERT_DATA_SQL_QUERY = """
    INSERT INTO top_scorers (position, player, club, total_goals, penalty_goals, assists, matches, mins, age)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (player, club) DO UPDATE SET
        total_goals = EXCLUDED.total_goals,
        penalty_goals = EXCLUDED.penalty_goals,
        assists = EXCLUDED.assists,
        matches = EXCLUDED.matches,
        mins = EXCLUDED.mins,
        age = EXCLUDED.age
    """
    # Convert DataFrame to list of tuples
    data_values_as_tuples = [tuple(row) for row in df.itertuples(index=False)]

    try:
        # Execute the query
        cursor.executemany(INSERT_DATA_SQL_QUERY, data_values_as_tuples)
        db_connection.commit()
        print("Data inserted or updated successfully ✅")

    except Error as e:
        print(f"❌ [INSERTING DATA ERROR]: '{e}'")

In [14]:
def run_data_pipeline():
    """
    Execute the ETL pipeline 
    """

data = get_top_scorers(url, headers, querystring)

if data and 'response' in data and data['response']:
    top_scorers = process_top_scorers(data)
    df = create_dataframe(top_scorers)
    print(df.to_string(index=False))

else:
    print("No data available or an error occurred ❌")

db_connection = create_db_connection(host, username, password, db)


# If connection is successful, proceed with creating table and inserting data
if db_connection is not None:
    create_table(db_connection)
    df = create_dataframe(top_scorers)
    insert_into_table(db_connection, df)

if __name__ == "__main__":
    run_data_pipeline()

 position           player              club  total_goals  penalty_goals  assists  matches  mins  age
        1          H. Kane         Tottenham           23              4       14       35  3087   30
        2    Mohamed Salah         Liverpool           22              6        5       37  3082   31
        3  Bruno Fernandes Manchester United           18              9       12       37  3109   29
        4    Son Heung-Min         Tottenham           17              1       10       37  3126   31
        4       P. Bamford             Leeds           17              2        7       38  3062   30
        5 D. Calvert-Lewin           Everton           16              0        0       33  2876   27
        6         J. Vardy         Leicester           15              8        9       34  2845   37
        7       O. Watkins       Aston Villa           14              1        5       37  3329   28
        8      İ. Gündoğan   Manchester City           13              1        2 

In [59]:
create_table(create_db_connection(host,username,password,db))

PostgreSQL Database connection successful ✅
Table created successfully ✅
